The stop frequency model assigns to each tour the number of intermediate destinations a person will travel to on each leg of the tour from the origin to tour primary destination and back. Intermediate stops are not modeled for drive-transit tours

In [1]:
import os
import toml
import pandas as pd
import numpy as np
import plotly.express as px
# import validation_data_input
import psrc_theme


import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

# run get_data.ipynb to load validation data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data
land_use = validation_data.land_use.copy()
per_data = validation_data.persons_data_uncloned.copy()
hh_data = validation_data.hh_data_uncloned.copy()
tour_data = validation_data.tours.copy()

In [3]:
tour_data['stop_frequency_out'] = tour_data['stop_frequency'].apply(lambda x: str(x)[0])
tour_data['stop_frequency_in'] = tour_data['stop_frequency'].apply(lambda x: str(x)[-3])

# add auto_ownership with 4+
hh_data['auto_ownership_simple'] = hh_data['auto_ownership'].apply(lambda x: "4+" if x>=4.0 else str(x))
# household density groups
var_group = land_use['log_hh_1'].quantile([.00, .125, .25, .50, .75,1.00])
land_use['household_density_bin'] = pd.cut(land_use['log_hh_1'], bins=var_group.tolist(),
                                           labels=['very low', 'low', 'medium', 'medium-high', 'high'])

# add person type labels
ptype_cat = {1: "1: Full-Time Worker",
             2: "2: Part-Time Worker",
             3: "3: University Student",
             4: "4: Non-Working Adult Age <65",
             5: "5: Non-Working Adult Age 65+",
             6: "6: High School Student Age 16+",
             7: "7: Child Age 5-15",
             8: "8: Child Age 0-4"}
per_data['ptype_label'] = per_data['ptype'].map(ptype_cat)

hh_data = hh_data.merge(land_use[['log_hh_1','household_density_bin','zone_id']],how="left",left_on='home_zone_id',right_on='zone_id')
per_data = per_data.merge(hh_data[['household_id','auto_ownership','auto_ownership_simple','log_hh_1','household_density_bin','source']],
                          how='left', on=['household_id','source']) # get auto ownership from hh data

tour_data = tour_data.merge(per_data, how='left', on=['person_id','household_id','source'])

In [4]:
#| layout-ncol: 2
df_plot = tour_data.groupby(['source','stop_frequency_out'])['tour_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="stop_frequency_out", y="percentage", color="source",barmode="group",
             title="outbound stop frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=400, yaxis=dict(tickformat=".1%"))
fig.show()

df_plot = tour_data.groupby(['source','stop_frequency_in'])['tour_weight'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="stop_frequency_in", y="percentage", color="source",barmode="group",
             title="inbound stop frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=400, yaxis=dict(tickformat=".1%"))
fig.show()

In [5]:
tour_data= tour_data.assign(stop_frequency_total = lambda x: x['stop_frequency_out'] + x['stop_frequency_in'])

df_plot2 = tour_data.groupby(['source','stop_frequency_total'])['tour_weight'].sum().reset_index()
df_plot2['percentage'] = df_plot2.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot2, x="stop_frequency_total", y="percentage", color="source",barmode="group",
             title="total stop frequency")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".1%"), xaxis = dict(dtick = 1))
fig.show()

## stop frequency by tour type

In [6]:
def gen_stopfreq_by_tourtype(df: pd.DataFrame, in_out: str, type_value: str):
    df_plot = df.groupby(['source','tour_type',in_out])['tour_weight'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source','tour_type'], group_keys=False)['tour_weight']. \
        apply(lambda x: x / float(x.sum()))
    df_plot['stop_freq_type'] = type_value
    return df_plot.rename(columns={in_out: 'frequency'})

df_plot = pd.concat([gen_stopfreq_by_tourtype(tour_data,'stop_frequency_out',"out"),
                     gen_stopfreq_by_tourtype(tour_data,'stop_frequency_in',"in"),
                     gen_stopfreq_by_tourtype(tour_data,'stop_frequency_total',"total")])

In [7]:
def plot_stop_frequency(df: pd.DataFrame, plt_tour_type: str):

    df_plot_out = df.loc[(df['tour_type']==plt_tour_type) & (df['stop_freq_type']=="out")]
    df_plot_in = df.loc[(df['tour_type']==plt_tour_type) & (df['stop_freq_type']=="in")]
    df_plot_total = df.loc[(df['tour_type']==plt_tour_type) & (df['stop_freq_type']=="total")]

    fig = px.bar(df_plot_out,
                 x='frequency', y="percentage", color="source",barmode="group",
                 title=plt_tour_type + ": outbound stop frequency")
    fig.update_layout(height=350, width=350, yaxis=dict(tickformat=".1%"))
    fig.show()

    fig = px.bar(df_plot_in,
                 x='frequency', y="percentage", color="source",barmode="group",
                 title=plt_tour_type + ": intbound stop frequency")
    fig.update_layout(height=350, width=350, yaxis=dict(tickformat=".1%"))
    fig.show()

    fig = px.bar(df_plot_total,
                 x='frequency', y="percentage", color="source",barmode="group",
                 title=plt_tour_type + ": total stop frequency")
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
    fig.update_layout(height=350, width=700, yaxis=dict(tickformat=".1%"), xaxis = dict(dtick = 1))
    fig.show()

In [8]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "business")

In [9]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "eat")

In [10]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "eatout")

In [11]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "escort")

In [12]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "maint")

In [13]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "othdiscr")

In [14]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "othmaint")

In [15]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "school")

In [16]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "shopping")

In [17]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "social")

In [18]:
#| layout: [[1,1], [1]]
plot_stop_frequency(df_plot, "work")